In [1]:
import pickle
import gzip
import numpy as np
import os
import matplotlib.pyplot as plt


In [2]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Preprocessing

In [ ]:
processed_data = load_zipped_pickle("/content/drive/MyDrive/data/train_processed.pkl")

In [ ]:
import numpy as np

def calculate_class_frequency(labels):
    """
    Calculate the frequency of each class in the given labels.
    Assuming binary classes with 0 representing the background and 1 the foreground.
    """
    background_count = 0
    foreground_count = 0

    # Iterate through each label (mask)
    for label in labels:
        # Flatten the label mask
        label_flat = label.flatten()

        # Count number of pixels for each class
        background_count += np.count_nonzero(label_flat == 0)
        foreground_count += np.count_nonzero(label_flat == 1)

    return background_count, foreground_count


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

videos = [entry['video'] for entry in processed_data]
labels = [entry['label'] for entry in processed_data]

height = 256
width = 256

X_frames = []
y_labels = []

for video, label in zip(videos, labels):
    num_frames = video.shape[-1]
    for i in range(num_frames):
        frame = video[:, :, i]
        label_frame = label[:, :, i]

        X_frames.append(frame)
        y_labels.append(label_frame)

X = np.array(X_frames).reshape(-1, height, width, 1)  # Adding channel dimension
y = np.array(y_labels).reshape(-1, height, width, 1)  # Adding channel dimension

print("X shape:", X.shape)
print("y shape:", y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
import numpy as np
import cv2
from scipy.ndimage import gaussian_filter, map_coordinates
from skimage import transform

np.random.seed(42)

def elastic_transform(image, label, alpha, sigma):
    random_state = np.random.RandomState(42)
    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), indexing="ij")
    indices = np.reshape(x + dx, (-1, 1)), np.reshape(y + dy, (-1, 1))
    distorted_image = map_coordinates(image, indices, order=1).reshape(shape)
    distorted_label = map_coordinates(label, indices, order=0).reshape(shape)
    return distorted_image, distorted_label

def apply_augmentation(image, label, image_size=(256, 256)):
    image = np.uint8(image * 255)
    label = np.uint8(label * 255)

    original_shape = image.shape[:2]

    # Apply elastic deformation with a certain probability
    if np.random.rand() < 0.1:
        alpha_range = (1, 2)
        sigma = 6
        alpha = np.random.uniform(*alpha_range)
        image, label = elastic_transform(image, label, alpha, sigma)

    # Apply scaling with a 50% probability
    if np.random.rand() < 0.5:
        scale_factor = np.random.uniform(0.8, 1.2)
        image = cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
        label = cv2.resize(label, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_NEAREST)
        image = cv2.resize(image, image_size, interpolation=cv2.INTER_LINEAR)
        label = cv2.resize(label, image_size, interpolation=cv2.INTER_NEAREST)

    # Apply rotation with a 30% probability
    if np.random.rand() < 0.3:
        angle = np.random.uniform(0, 30)
        center = (original_shape[1] // 2, original_shape[0] // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        image = cv2.warpAffine(image, rotation_matrix, original_shape)
        label = cv2.warpAffine(label, rotation_matrix, original_shape)

    # Apply flipping with a 30% probability
    if np.random.rand() < 0.3:
        flip_horizontal = np.random.rand() < 0.5
        if flip_horizontal:
            image = cv2.flip(image, 1)  # Horizontal flip
            label = cv2.flip(label, 1)
        else:
            image = cv2.flip(image, 0)  # Vertical flip
            label = cv2.flip(label, 0)

    image = image.astype(np.float32) / 255.0
    label = label.astype(np.float32) / 255.0
    image = image.reshape(image_size + (1,))
    label = label.reshape(image_size + (1,))

    return image, label



In [ ]:
# Apply augmentations with frequency adjustment to the training data
X_train_augmented = []
y_train_augmented = []

for frame, label in zip(X_train, y_train):
    # Apply augmentation with a given probability
    frame = np.squeeze(frame)
    label = np.squeeze(label)
    augmented_frame, augmented_label = apply_augmentation(frame, label)

    augmented_frame = np.expand_dims(augmented_frame.astype(np.float32), axis=-1)
    augmented_label = np.expand_dims(augmented_label.astype(np.float32), axis=-1)

    X_train_augmented.append(augmented_frame)
    y_train_augmented.append(augmented_label)

# Convert augmented lists to numpy arrays
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

In [ ]:
# Assuming y_train is your array of training labels (masks)
background_count, foreground_count = calculate_class_frequency(y_train_augmented)

print(f"Background Pixels: {background_count}")
print(f"Foreground Pixels: {foreground_count}")
total_pixels = background_count + foreground_count

# Calculate weights
weight_for_background = (1 / background_count) * (total_pixels) / 2.0
weight_for_foreground = (1 / foreground_count) * (total_pixels) / 2.0

class_weights = {0: weight_for_background, 1: weight_for_foreground}
print(class_weights)


In [ ]:
'''
total_weight = weight_for_background + weight_for_foreground
weight_for_background = weight_for_background / total_weight
weight_for_foreground = weight_for_foreground / total_weight

print("Weight for Background:", weight_for_background)
print("Weight for Foreground:", weight_for_foreground)
'''


In [ ]:
import matplotlib.pyplot as plt

def plot_processed_frame(frame_index, X_data, y_data):
    plt.figure(figsize=(12, 6))

    # Original image and label
    original_image = X_data[frame_index].squeeze()
    original_label = y_data[frame_index].squeeze()

    # Plot original image
    plt.subplot(2, 2, 1)
    plt.imshow(original_image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')

    # Plot original label
    plt.subplot(2, 2, 2)
    plt.imshow(original_label, cmap='gray')
    plt.title('Original Label')
    plt.axis('off')

    # Apply augmentation
    augmented_image, augmented_label = apply_augmentation(original_image, original_label)

    # Plot augmented image
    plt.subplot(2, 2, 3)
    plt.imshow(augmented_image.squeeze(), cmap='gray')
    plt.title('Augmented Image')
    plt.axis('off')

    # Plot augmented label
    plt.subplot(2, 2, 4)
    plt.imshow(augmented_label.squeeze(), cmap='gray')
    plt.title('Augmented Label')
    plt.axis('off')

    plt.show()

frame_index = 100
plot_processed_frame(frame_index, X_train, y_train)


## Model REB Unet

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, UpSampling2D, concatenate, GlobalAveragePooling2D, Dense, Reshape, multiply, Add

def conv_block(input_tensor, num_filters):
    # Apply two convolutional layers with ReLU activation and batch normalization
    x = Conv2D(num_filters, (3, 3), padding="same")(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

def residual_block(input_tensor, num_filters):
    # Residual block with skip connection
    conv = conv_block(input_tensor, num_filters)
    skip = Conv2D(num_filters, (1, 1), padding='same')(input_tensor)
    skip = BatchNormalization()(skip)
    output = Add()([conv, skip])
    return output

def SEBlock(input_tensor, num_filters, ratio=16):
    # Squeeze and Excitation Block
    squeeze = GlobalAveragePooling2D()(input_tensor)
    excitation = Dense(num_filters // ratio, activation='relu')(squeeze)
    excitation = Dense(num_filters, activation='sigmoid')(excitation)
    excitation = Reshape((1, 1, num_filters))(excitation)
    scale = multiply([input_tensor, excitation])
    return scale

def ASPP(input_tensor, num_filters, rate=[6, 12, 18]):
    # ASPP block with different dilation rates
    aspp1 = Conv2D(num_filters, (3, 3), dilation_rate=rate[0], padding='same')(input_tensor)
    aspp1 = BatchNormalization()(aspp1)
    aspp1 = Activation('relu')(aspp1)

    aspp2 = Conv2D(num_filters, (3, 3), dilation_rate=rate[1], padding='same')(input_tensor)
    aspp2 = BatchNormalization()(aspp2)
    aspp2 = Activation('relu')(aspp2)

    aspp3 = Conv2D(num_filters, (3, 3), dilation_rate=rate[2], padding='same')(input_tensor)
    aspp3 = BatchNormalization()(aspp3)
    aspp3 = Activation('relu')(aspp3)

    # Concatenate the atrous convolution outputs
    aspp = concatenate([aspp1, aspp2, aspp3], axis=-1)

    # 1x1 convolution to combine features
    aspp = Conv2D(num_filters, (1, 1), padding='same')(aspp)
    aspp = BatchNormalization()(aspp)
    aspp = Activation('relu')(aspp)

    return aspp


def create_model(input_shape):
    inputs = Input(input_shape)

    # Encoder with residual and SE blocks
    x = residual_block(inputs, 64)
    x = SEBlock(x, 64)
    x = MaxPooling2D((2, 2))(x)

    x = residual_block(x, 128)
    x = SEBlock(x, 128)
    x = MaxPooling2D((2, 2))(x)

    x = residual_block(x, 256)
    x = SEBlock(x, 256)
    x = MaxPooling2D((2, 2))(x)

    # Bridge with ASPP
    x = ASPP(x, 512)

    # Decoder with residual and SE blocks
    x = UpSampling2D((2, 2))(x)  # Upsampling to 64x64
    x = residual_block(x, 256)
    x = SEBlock(x, 256)

    x = UpSampling2D((2, 2))(x)  # Upsampling to 128x128
    x = residual_block(x, 128)
    x = SEBlock(x, 128)

    x = UpSampling2D((2, 2))(x)  # Upsampling to 256x256
    x = residual_block(x, 64)
    x = SEBlock(x, 64)

    # Final Convolution Layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)  # Final layer with shape (256, 256, 1)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model




In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, UpSampling2D, concatenate, GlobalAveragePooling2D, Dense, Reshape, multiply

def conv_block(input_tensor, num_filters, use_BN=True):
    # First layer
    x = Conv2D(num_filters, (3, 3), padding="same")(input_tensor)
    if use_BN:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Second layer
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    if use_BN:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input_tensor, num_filters, use_BN=True):
    x = conv_block(input_tensor, num_filters, use_BN)
    p = MaxPooling2D((2, 2))(x)
    return x, p

def decoder_block(input_tensor, concat_tensor, num_filters, use_BN=True):
    x = UpSampling2D((2, 2))(input_tensor)
    x = concatenate([x, concat_tensor])
    x = conv_block(x, num_filters, use_BN)
    return x

def SEBlock(input_tensor, num_filters, ratio=16):
    squeeze = GlobalAveragePooling2D()(input_tensor)
    excitation = Dense(num_filters // ratio, activation='relu')(squeeze)
    excitation = Dense(num_filters, activation='sigmoid')(excitation)
    excitation = Reshape((1, 1, num_filters))(excitation)
    scale = multiply([input_tensor, excitation])
    return scale

def unet_model(input_shape, num_filters=64, use_BN=True):
    inputs = Input(input_shape)

    # Encoder
    x1, p1 = encoder_block(inputs, num_filters, use_BN)
    x1 = SEBlock(x1, num_filters)
    x2, p2 = encoder_block(p1, num_filters*2, use_BN)
    x2 = SEBlock(x2, num_filters*2)
    x3, p3 = encoder_block(p2, num_filters*4, use_BN)
    x3 = SEBlock(x3, num_filters*4)
    x4, p4 = encoder_block(p3, num_filters*8, use_BN)
    x4 = SEBlock(x4, num_filters*8)

    # Bridge
    bridge = conv_block(p4, num_filters*16, use_BN)
    bridge = SEBlock(bridge, num_filters*16)

    # Decoder
    d1 = decoder_block(bridge, x4, num_filters*8, use_BN)
    d2 = decoder_block(d1, x3, num_filters*4, use_BN)
    d3 = decoder_block(d2, x2, num_filters*2, use_BN)
    d4 = decoder_block(d3, x1, num_filters, use_BN)

    # Output
    outputs = Conv2D(1, (1, 1), activation="sigmoid")(d4)

    model = Model(inputs, outputs)
    return model




In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

input_shape = (256, 256, 1)


#unet = create_model(input_shape)
unet = unet_model(input_shape)
lr=1e-3

reduce_lr_plateau = ReduceLROnPlateau(monitor='val_loss',
                                      factor=0.2,   # Reduction factor (new_lr = lr * factor).
                                      patience=15,   # Number of epochs with no improvement after which learning rate will be reduced.
                                      min_lr=1e-6,  # Lower bound on the learning rate.
                                      verbose=1)

optimizer = Adam(learning_rate=lr)


callbacks = [EarlyStopping(monitor='val_loss', patience=25),
                 ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True),
             reduce_lr_plateau
             ]


#unet.summary()

In [ ]:
unet.summary()

In [ ]:
import numpy as np
from tensorflow.keras import backend as K

def jaccard_index(y_true, y_pred):
    """Calculates Jaccard index (IoU) for each image in a batch and averages them."""
    y_true_f = K.batch_flatten(K.cast(y_true, 'float32'))
    y_pred_f = K.batch_flatten(K.cast(y_pred, 'float32'))
    intersection = K.sum(y_true_f * y_pred_f, axis=-1)
    union = K.sum(y_true_f, axis=-1) + K.sum(y_pred_f, axis=-1) - intersection
    return K.mean((intersection + K.epsilon()) / (union + K.epsilon()))



In [ ]:
def power_jaccard_loss(y_true, y_pred, p=2, epsilon=1e-7):
    y_true_f = K.batch_flatten(K.cast(y_true, 'float32'))
    y_pred_f = K.batch_flatten(K.cast(y_pred, 'float32'))
    intersection = K.sum(y_true_f * y_pred_f, axis=-1)
    union = K.sum(K.pow(y_true_f, p), axis=-1) + K.sum(K.pow(y_pred_f, p), axis=-1) - intersection
    return 1 - K.mean((intersection + epsilon) / (union + epsilon))



In [ ]:
def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(K.cast(y_true, 'float32'))
    y_pred_f = K.flatten(K.cast(y_pred, 'float32'))
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


In [ ]:
def tversky_loss(y_true, y_pred, alpha=0.5, beta=0.5, smooth=1e-6):
    y_true_f = K.flatten(K.cast(y_pred, 'float32'))
    y_pred_f = K.flatten(K.cast(y_pred, 'float32'))
    truepos = K.sum(y_true_f * y_pred_f)
    fp_and_fn = alpha * K.sum(y_pred_f * (1 - y_true_f)) + beta * K.sum((1 - y_pred_f) * y_true_f)
    return 1 - (truepos + smooth) / (truepos + fp_and_fn + smooth)


In [ ]:
import tensorflow as tf
from tensorflow.keras.losses import binary_crossentropy
import tensorflow.keras.backend as K

unet = create_model((256, 256, 1))

unet.compile(optimizer=optimizer,
             loss=tversky_loss,
             metrics=[jaccard_index])

history = unet.fit(
    X_train_augmented, y_train_augmented,
    batch_size=8,
    epochs=100,
    validation_data = (X_test,y_test),
    callbacks=callbacks
)



In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['jaccard_index'])
    plt.plot(history.history['val_jaccard_index'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')

    plt.show()

plot_training_history(history)


In [ ]:
import numpy as np
from scipy import ndimage

def post_process_segmentation(seg_map, threshold=0.5, min_size=100):
    """
    Post-processes the predicted segmentation map.

    Parameters:
    - seg_map: 2D numpy array, the predicted segmentation map.
    - threshold: float, the threshold to apply to the segmentation map to binarize.
    - min_size: int, the minimum size of connected components to keep.

    Returns:
    - processed_seg_map: 2D numpy array, the processed segmentation map.
    """
    # Threshold the segmentation map
    seg_map = (seg_map > threshold).astype(np.float32)

    # Label connected components
    labeled_seg_map, num_features = ndimage.label(seg_map)

    # Remove small objects
    unique, counts = np.unique(labeled_seg_map, return_counts=True)
    for (label, size) in zip(unique, counts):
        if size < min_size:
            labeled_seg_map[labeled_seg_map == label] = 0

    # Recreate binary seg map
    processed_seg_map = (labeled_seg_map > 0).astype(np.float32)

    return processed_seg_map


In [ ]:
import numpy as np
from tensorflow.keras import backend as K

# Predict on test set
y_pred = unet.predict(X_test)
y_pred = unet.predict(X_test)

# Apply post-processing to each prediction in the batch
y_pred_processed = np.array([post_process_segmentation(pred, threshold=0.5, min_size=100) for pred in y_pred])

# Convert predictions to binary format
y_pred_binary = (y_pred_processed > 0.5).astype(np.float32)

# Calculate Jaccard Index for each image in the test set
jaccard_values = np.array([jaccard_index(y_test[i], y_pred_binary[i]) for i in range(len(y_test))])
jaccard_indices = K.eval(jaccard_values)

# Compute median Jaccard Index
median_jaccard = np.median(jaccard_indices)
print(f"Median Jaccard Index: {median_jaccard}")


In [ ]:
import matplotlib.pyplot as plt

# Select the first image from X_test
sample_index = 33
sample_image = X_test[sample_index]

true_label = y_test[sample_index]
predicted_label = y_pred_binary[sample_index]

# Plot the original image
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.imshow(sample_image.squeeze(), cmap='gray')
plt.title("Original Image")
plt.axis('off')

# Plot the true label
plt.subplot(132)
plt.imshow(true_label.squeeze(), cmap='gray')
plt.title("True Label")
plt.axis('off')

# Plot the predicted label
plt.subplot(133)
plt.imshow(predicted_label.squeeze(), cmap='gray')
plt.title("Predicted Label")
plt.axis('off')

plt.show()


### Real Test predictions

In [ ]:
real_test_data = load_zipped_pickle("/content/drive/MyDrive/data/test_processed.pkl")


In [ ]:
real_test_data_not_processed = load_zipped_pickle("/content/drive/MyDrive/data/test.pkl")

In [ ]:
import numpy as np
import cv2

predictions = []

for processed_entry, original_entry in zip(real_test_data, real_test_data_not_processed):
    video_frames = processed_entry['video']
    num_frames = video_frames.shape[2]
    prediction_frames = []

    original_height, original_width = original_entry['video'].shape[:2]

    for i in range(num_frames):
        frame = video_frames[:, :, i]
        frame_reshaped = np.expand_dims(frame, axis=0)
        frame_reshaped = np.expand_dims(frame_reshaped, axis=-1)

        # Predict mask
        pred_mask = unet.predict(frame_reshaped)
        pred_binary_mask = (pred_mask > 0.5).astype(np.uint8)

        pred_resized = cv2.resize(pred_binary_mask[0, :, :, 0], (original_width, original_height), interpolation=cv2.INTER_NEAREST)
        pred_resized = pred_resized.astype(np.bool)
        prediction_frames.append(pred_resized)

    prediction = np.stack(prediction_frames, axis=-1)

    # Data Structure
    predictions.append({
        'name': processed_entry['name'],
        'prediction': prediction
    })

# Save predictions in the correct format
save_zipped_pickle(predictions, 'predictions_unet_se.pkl')


In [ ]:
import matplotlib.pyplot as plt

def plot_image_and_mask(image, mask):
    plt.figure(figsize=(12, 6))

    # Plot the original image
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title("Original Image")
    plt.axis('off')

    # Plot the original image with mask overlay
    plt.subplot(1, 2, 2)
    plt.imshow(image, cmap='gray')
    plt.imshow(mask, cmap='jet', alpha=0.5)  # Overlay with transparency
    plt.title("Image with Mask Overlay")
    plt.axis('off')

    plt.show()

# Select video 10 and image 1
video_index = 15
image_index = 2

# Access the original frame and the predicted mask
original_frame = real_test_data_not_processed[video_index]['video'][:, :, image_index]
predicted_mask = predictions[video_index]['prediction'][:, :, image_index]

# Plotting the original image and its mask
plot_image_and_mask(original_frame, predicted_mask)


In [ ]:
#sample_data = load_zipped_pickle("/content/drive/MyDrive/data/sample.pkl")
